In [1]:
import pandas as pd
import pymysql

def dev_db_conn():
    conn = pymysql.connect( 
        host = 호스트, 
        user = 유저, 
        password = 비밀번호, 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn


def live_db_conn(): 
    conn = pymysql.connect( 
        host = 호스트, 
        user = 유저, 
        password = 비밀번호, 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = DB명) 
    return conn


def country_confirm(email_text, country):
    """이메일을 통해 국적 자동화 함수"""
    try:
        domain = email_text.split("@")[-1]
        domain_detail = "." + email_text.split("@")[-1].split(".")[-1]

        if domain_detail in dic_domain_list:
            return dic_domain[dic_domain["ccTLD"] == domain_detail]["use_nm"].reset_index(drop=True)[0]

        elif domain in dic_homepage_list:
            return dic_homepage[dic_homepage["ccTLD"] == domain]["use_nm"].reset_index(drop=True)[0]

        return country
    except:
        return country

In [2]:
with live_db_conn() as conn:
    sql = """
    SELECT T1.userId, T1.cardNo, T4.name AS 'country', T3.email, T5.id AS 'KONA_CARD_ID'
    FROM DB명.KONA_TRANSACTION T1
    LEFT JOIN DB명.KTC_USER T2 ON T1.userId = T2.card_user_id AND T2.`type` = 1
    LEFT JOIN DB명.user T3 ON T2.user_id = T3.id
    LEFT JOIN DB명.IA_KTC_COUNTRY_CODE T4 ON T3.country = T4.code AND T4.lang = 'ko'
    INNER JOIN DB명.KONA_CARD T5 ON T1.cardNo = T5.card_no
    WHERE T1.userId != 0
    GROUP BY T1.userId, T1.cardNo
    ORDER BY T1.id ASC
    """
    KONA_USER = pd.read_sql(sql, conn)

with dev_db_conn() as conn:
    sql = """
    SELECT ccTLD, use_nm
    FROM dictionary.dic_domain
    WHERE `status` = 'normal' AND `type` = 'domain' AND ccTLD != '.kr';
    """
    dic_domain = pd.read_sql(sql, conn)
    
    sql = """
    SELECT ccTLD, use_nm
    FROM dictionary.dic_domain
    WHERE `status` = 'normal' AND `type` = 'homepage';
    """
    dic_homepage = pd.read_sql(sql, conn)
    
dic_domain_list = dic_domain["ccTLD"].tolist()
dic_homepage_list = dic_homepage["ccTLD"].tolist()

KONA_USER["NLTY"] = KONA_USER.apply(lambda row:(country_confirm(row["email"], row["country"])), axis=1).tolist()

with live_db_conn() as conn:
    with conn.cursor() as curs:
        sql = """
        UPDATE redtable2021.KONA_SOURCE T1
        SET T1.country = %s
        WHERE T1.KONA_CARD_ID = %s
        """
        val = KONA_USER.apply(lambda row:(row["NLTY"], row["KONA_CARD_ID"]), axis=1).tolist()
        curs.executemany(sql, val)